In [1]:
# Notebook tests for tar aliases (e.g., sort-tar-archive, tar-this-dir-dated, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
#Global Setup

TMP=/tmp/temp-aliases-test
TAR_TEMP=$HOME/temp/tarFolder

mkdir $TMP --parents
command cd $TMP
mkdir tarFolder/first/second/third --parents
mkdir tarFolder/temp

In [3]:
# extract-tar tests
# create tar to later extract

echo "logs" > tarFolder/temp/temp.log
TXT=tarFolder/first/second/third/tar.txt
echo "tar this" > $TXT
tar -zcvf tared.tar.gz tarFolder > /dev/null
rm -rf tarFolder > /dev/null

In [4]:
# extract the tar just created

extract-tar tared.tar.gz > /dev/null

In [5]:
# Assert the files are extracted

[ $(ls | grep Folder | wc -l) -eq 1 ]; echo $?;

0


In [6]:
#extract-tar-force tests

echo "tar this 2" >> $TXT
prev_lines=$(cat $TXT | wc -l )
extract-tar-force tared.tar.gz > /dev/null

In [7]:
#Assert amount of lines after and before extract-force

[ $prev_lines -eq 2 ] && [ $(cat $TXT | wc -l) -eq 1 ]; echo $?;

0


In [8]:
# un-tar tests
# same as extract-tar

rm -rf tarFolder > /dev/null
un-tar tared.tar.gz > /dev/null

In [9]:
# Assert the files are extracted

[ $(ls | grep tarFolder | wc -l) -eq 1 ]; echo $?;

0


In [10]:
#extract-tar-here tests

mkdir testFolder
command cd $TMP/testFolder
extract-tar-here tared.tar.gz > /dev/null

In [11]:
# Assert the files are extracted

command cd $TMP
[ $(ls | grep testFolder | wc -l) -eq 1 ]; echo $?;

0


In [12]:
# view-tar tests
# Assert the existence of five dirs inside tar

tared=$(view-tar tared.tar.gz | wc -l)
[ $tared -eq 7 ]; echo $?;

0


In [13]:
# make-tar tests
# test with no optional args

make-tar test tarFolder > /dev/null
test=$(view-tar test.tar.gz | wc -l)

In [14]:
# assert lines_test should be equal to lines_tared

[ $tared -eq $test ]; echo $?

0


In [15]:
rm -f test.tar.gz
# make-tar with arguments ignores empty dirs

make-tar test tarFolder 5 > /dev/null
test=$(view-tar test.tar.gz | wc -l)


removed 'test.tar.gz'


In [16]:
# assert theres only one line

[ $test -eq 2 ]; echo $?

0


In [17]:
#if no optional arguments are given
# make-tar skips filtering and a few other steps, 
# so it should execute less commands overall

filtered=$(trace-cmd make-tar test dir 5 | wc -l) 
unfiltered=$(trace-cmd make-tar test dir | wc -l) 

In [18]:
[ $filtered -gt $unfiltered ]; echo $?;

0


In [19]:
# tar-dir tests

command cd $HOME
tar-dir $TMP/tarFolder > /dev/null

In [20]:
# lines_tar_dir should also be equal to lines_tared, but since it uses 
# make-tar it probably shouldn't

tar_dir=$(view-tar $TEMP/tarFolder.tar.gz | wc -l)
[ $tared -eq $tar_dir ]; echo $?;

0


In [21]:
# tar-just-dir tests
# tar-just-dir tars the given dir with specified depth of 1

tar-just-dir $TMP/tarFolder/first/second/third > /dev/null
tar-just-dir $TMP/tarFolder > /dev/null
not_empty=$(view-tar $HOME/temp/third.tar.gz | wc -l)
empty=$(view-tar $TAR_TEMP.tar.gz | wc -l)

In [22]:
#assert lines_empty equals 0 and lines_not_empty is not (should equals 1)

[ $not_empty -gt $empty ] && [ $empty -eq 0 ]; echo $?;

0


In [23]:
# tar-this-dir tests

command cd $TMP/tarFolder
tar-this-dir > /dev/null
this_dir=$(view-tar $TAR_TEMP.tar.gz | wc -l )

In [24]:
# assert the .tar includes all subdirs and files in tarFolder

[ $this_dir -eq 7 ]; echo $?;

0


In [25]:
# new-tar-this-dir tests
# NOTE: THIS PRODUCES A TAR FILE WITHOUT NAME

rm -f $HOME/temp/tarFolder.tar.gz
new-tar-this-dir > /dev/null
new_this_dir=$(view-tar $HOME/temp/tarFolder.tar.gz | wc -l)

removed '/home/lolo/temp/tarFolder.tar.gz'


In [26]:
#assert new-tar-this-dir produced the tar with the correct name and files inside
[ $new_this_dir -eq 7 ]; echo $?;

1


In [27]:
# tar-this-dir-dated tests
tar-this-dir-dated > /dev/null
this_dir_dated=$(view-tar $TAR_TEMP-$(TODAY).tar.gz | wc -l )
rm -f $TAR_TEMP-$(TODAY).tar.gz

removed '/home/lolo/temp/tarFolder-27dic23.tar.gz'


In [28]:
#assert tar-this-dir-dated 
[ $this_dir_dated -eq $this_dir ]; echo $?;

0


In [29]:
# tar-this-dir-normal tests
# normal excludes temp, backup and archive folders

#DOESN'T WORK BECAUSE DEFINITION OF TAR-DIR IS WRONG

tar-this-dir-normal > /dev/null
this_dir_normal=$(view-tar $TAR_TEMP.tar.gz | wc -l )

In [30]:
[ $this_dir_normal -eq 1 ]; echo $?

1


In [31]:
# tar-just-this-dir tests

# since tarFolder just contains folders, and tarFolder/temp contains one file,
# tarFolder_plain = 0 and tarFolder_temp = 1 should be true
command cd $TMP/tarFolder
tar-just-this-dir > /dev/null
tarFolder_plain=$(view-tar $HOME/temp/tarFolder.tar.gz | wc -l)

command cd temp
tar-just-this-dir > /dev/null
tarFolder_temp=$(view-tar $HOME/temp/temp.tar.gz | wc -l)

/tmp/temp-aliases-test/tarFolder/temp


In [32]:
# assert the results are correct
[ $tarFolder_plain -eq 0 ] && [ $tarFolder_temp -eq 1 ]; echo $?;

0


In [33]:
# tar-just-this-dir-dated tests

# should produce same results as not dated version

command cd $TMP/tarFolder
tar-just-this-dir-dated > /dev/null
tarFolder_plain_dated=$(view-tar $HOME/temp/tarFolder-$(TODAY).tar.gz | wc -l)

command cd temp
tar-just-this-dir-dated > /dev/null
tarFolder_temp_dated=$(view-tar $HOME/temp/temp-$(TODAY).tar.gz | wc -l)

/tmp/temp-aliases-test/tarFolder/temp


In [34]:
# assert the results are consistent with the non-dated version

[ $tarFolder_plain_dated -eq $tarFolder_plain ] && [ $tarFolder_temp_dated -eq $tarFolder_temp ]; echo $?; 

0


In [35]:
# make-recent-tar tests
# make-recent-tar - 0 is for files that have been just created, otherwise it specifies less than how many days old the files can be, ignoring folders
command cd $TMP/tarFolder
make-recent-tar test.tar.gz 1 > /dev/null
recent_tar=$(view-tar test.tar.gz | wc -l)

In [36]:
# assert make-recent-tar takes both files inside tarFolder
[ $recent_tar -eq 2 ]; echo $?;

0


In [37]:
# recent-tar-this-dir tests
# should make the same result as make-recent-tar

# make-recent-tar DOESNT INCLUDE .tar.gz in the file name
rm -f test.tar.gz
recent-tar-this-dir 1 > /dev/null
recent_this_dir_one=$(view-tar ~/temp/recent-tarFolder.tar.gz | wc -l)



removed 'test.tar.gz'


In [38]:
#assert there are two files created recently
[ $recent_this_dir_one -eq 2 ]; echo $?;

1


In [39]:
command cd $HOME
[ $TMP == /tmp/temp-aliases-test ] && rm -rf $TMP
command cd temp 
rm -f $(ls | grep tar)

removed '/tmp/temp-aliases-test/tarFolder/temp/temp.log'
removed directory '/tmp/temp-aliases-test/tarFolder/temp'
removed '/tmp/temp-aliases-test/tarFolder/first/second/third/tar.txt'
removed directory '/tmp/temp-aliases-test/tarFolder/first/second/third'
removed directory '/tmp/temp-aliases-test/tarFolder/first/second'
removed directory '/tmp/temp-aliases-test/tarFolder/first'
removed directory '/tmp/temp-aliases-test/tarFolder'
removed '/tmp/temp-aliases-test/test.tar.log'
removed '/tmp/temp-aliases-test/tared.tar.gz'
removed directory '/tmp/temp-aliases-test/testFolder'
removed '/tmp/temp-aliases-test/test.tar.gz'
removed directory '/tmp/temp-aliases-test'
/home/lolo/temp
removed 'recent-tarFolder'
removed 'tarFolder-27dic23.tar.gz'
removed 'tarFolder-27dic23.tar.log'
removed 'tarFolder-27dic23.tar.log-2'
removed 'tarFolder.tar.gz'
removed 'tarFolder.tar.log'
removed 'temp-27dic23.tar.gz'
removed 'temp-27dic23.tar.log'
removed 'temp.tar.gz'
removed 'temp.tar.log'
removed 'third.tar